In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
# from textblob import TextBlob

def preprocessing(lines):
    """
        Ingest a line of tweet and pre-processing it to get rid of improper characters
        TODO: enhance this function for generalization
    """
    words = lines.select(explode(split(lines.value, "t_end")).alias("word"))
    words = words.na.replace('', None)
    words = words.na.drop()
    words = words.withColumn('word', F.regexp_replace('word', r'http\S+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '@\w+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '#', ''))
    words = words.withColumn('word', F.regexp_replace('word', 'RT', ''))
    words = words.withColumn('word', F.regexp_replace('word', ':', ''))
    return words

In [2]:
# text classification
def polarity_detection(text):
    return TextBlob(text).sentiment.polarity
def subjectivity_detection(text):
    return TextBlob(text).sentiment.subjectivity
def text_classification(words):
    # polarity detection
    polarity_detection_udf = udf(polarity_detection, StringType())
    words = words.withColumn("polarity", polarity_detection_udf("word"))
    # subjectivity detection
    subjectivity_detection_udf = udf(subjectivity_detection, StringType())
    words = words.withColumn("subjectivity", subjectivity_detection_udf("word"))
    return words

In [3]:
# create Spark session
# read the tweet data received from socket (delivered by the tweeter API)
spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()
lines = spark.readStream.format("socket").option("host", "127.0.0.1").option("port", 5555).load()

# Preprocess the data
words = preprocessing(lines)

In [4]:
# # text classification to define polarity and subjectivity
words = text_classification(words)
words = words.repartition(1)
query = words.writeStream.queryName("all_tweets_2")\
    .outputMode("append").format("parquet")\
    .option("path", "./parc")\
    .option("checkpointLocation", "./check")\
    .trigger(processingTime='60 seconds').start()
query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [5]:
df = spark.read.parquet("./parc")
df.show()

+----+
|word|
+----+
+----+



In [10]:
df.info()

AttributeError: 'DataFrame' object has no attribute 'info'